# Henry's model select demo

The follow is the model select program base on Decision Tree Classfier.  
Will be run on .py file instead of Jupyter Noteboook (for CSE server)

In [2]:
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.base import TransformerMixin
from sklearn import tree
from sklearn import preprocessing
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals import joblib
import matplotlib.pyplot as plt

## Pre-define basic parameter for system adjustment

In [ ]:
np.random.seed(1)

TRAININGFILE = '../keyword.csv'
TESTFILE = '../key_word_test.csv'

x_label = ['article_words', 'key_word_20', 'key_word_50', 'key_word_100']
y_label = ['topic']

topic_code = {
    'ARTS CULTURE ENTERTAINMENT': 1,
    'BIOGRAPHIES PERSONALITIES PEOPLE': 2,
    'DEFENCE': 3,
    'DOMESTIC MARKETS': 4,
    'FOREX MARKETS': 5,
    'HEALTH': 6,
    'MONEY MARKETS': 7,
    'SCIENCE AND TECHNOLOGY': 8,
    'SHARE LISTINGS': 9,
    'SPORTS': 10,
    'IRRELEVANT': 0
}

## Pre-processing the data_set from the CSV file

In [ ]:
def preprocess(df, x_label, y_label):

    return df[x_label+y_label]

# for the bag of word and label encode process
def convert_word(bag_of_word_model, label_model, data_set):
    act_x = bag_of_word_model.transform(data_set[x_label].values)
    act_y = label_model.transform(data_set['topic'])

    return act_x, act_y